# Video tutorial
[How to Download Kline Data from Exchange (part I)](https://youtu.be/JwAK892TzT0)

In [1]:
import sys
sys.path.append("../lib")
from BinanceAPI import *
from datetime import datetime
import pandas as pd
import numpy as np
import time
import os

# Global variables

In [2]:
api_key, api_secret = "", ""
exchange = "Binance"
symbol = "BTCUSDT"

# Load kucoin API

In [3]:
api = BinanceAPI(api_key, api_secret)

# Util function

In [4]:
def toDatetime(ts):
    return datetime.fromtimestamp(ts)

def toTimestamp(dt):
    dt = datetime.strptime(dt, "%Y-%m-%d %H:%M:%S")
    return datetime.timestamp(dt)

# Load klines data

In [5]:
# Example responses from API documentation
# [
#   [
#     1499040000000,      // Open time
#     "0.01634790",       // Open
#     "0.80000000",       // High
#     "0.01575800",       // Low
#     "0.01577100",       // Close
#     "148976.11427815",  // Volume
#     1499644799999,      // Close time
#     "2434.19055334",    // Quote asset volume
#     308,                // Number of trades
#     "1756.87402397",    // Taker buy base asset volume
#     "28.46694368",      // Taker buy quote asset volume
#     "17928899.62484339" // Ignore.
#   ]
# ]

In [5]:
# get klines
etimestr = "2022-04-30 23:00:00"
etime = int(toTimestamp(etimestr)) * 1000
stime = etime - 3600 * 24 * 30 * 1000 + 1
klines = api.getKlines(symbol, "1h", stime=stime, etime=etime, limit=1000)

In [6]:
klines

[[1648742400000,
  '46453.81000000',
  '46492.65000000',
  '45669.00000000',
  '45688.80000000',
  '6315.81153000',
  1648745999999,
  '290514278.28878110',
  143756,
  '2779.39695000',
  '127852058.44726120',
  '0'],
 [1648746000000,
  '45688.80000000',
  '45891.94000000',
  '45651.35000000',
  '45820.00000000',
  '2363.95325000',
  1648749599999,
  '108247373.69161410',
  64559,
  '1225.24580000',
  '56103971.15937060',
  '0'],
 [1648749600000,
  '45819.99000000',
  '45962.08000000',
  '45595.28000000',
  '45957.27000000',
  '1822.32955000',
  1648753199999,
  '83456669.47258360',
  54649,
  '946.31150000',
  '43341485.41776020',
  '0'],
 [1648753200000,
  '45957.27000000',
  '45987.94000000',
  '45500.00000000',
  '45581.97000000',
  '2165.48834000',
  1648756799999,
  '99001442.66528660',
  56751,
  '903.94755000',
  '41319029.79505010',
  '0'],
 [1648756800000,
  '45581.98000000',
  '45790.00000000',
  '45519.03000000',
  '45745.46000000',
  '1550.11357000',
  1648760399999,
  '70

In [7]:
len(klines)

720

In [8]:
toDatetime(int(klines[-1][0]/1000))

datetime.datetime(2022, 4, 30, 23, 0)

In [9]:
toDatetime(int(klines[0][0]/1000))

datetime.datetime(2022, 4, 1, 0, 0)

# Pandas dataframe

In [10]:
def computeDF(klines):
    klines = [[pd.Timestamp(int(x[0]/1000), unit='s', tz='Asia/Hong_Kong'), \
            float(x[1]), \
            float(x[2]), \
            float(x[3]), \
            float(x[4]), \
            float(x[5]), \
            float(x[6]), \
            float(x[7]), \
            float(x[8]), \
            float(x[9]), \
            float(x[10])] \
            for x in klines]
    df = pd.DataFrame(np.array(klines), \
        columns=['Timestamp', 'open', 'high', 'low', 'close', \
        'volume', 'close_time', 'quote_asset_volume', 'num_trades', \
        'take_buy_base_vol', 'take_buy_quote_vol'])
    df = df.set_index("Timestamp") 
    return df 

In [11]:
df = computeDF(klines)

In [12]:
df.head(10)

,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,take_buy_base_vol,take_buy_quote_vol
Timestamp,,,,,,,,,,
2022-04-01 00:00:00+08:00,46453.8,46492.7,45669,45688.8,6315.81,1.64875e+12,2.90514e+08,143756,2779.4,1.27852e+08
2022-04-01 01:00:00+08:00,45688.8,45891.9,45651.3,45820,2363.95,1.64875e+12,1.08247e+08,64559,1225.25,5.6104e+07
2022-04-01 02:00:00+08:00,45820,45962.1,45595.3,45957.3,1822.33,1.64875e+12,8.34567e+07,54649,946.312,4.33415e+07
2022-04-01 03:00:00+08:00,45957.3,45987.9,45500,45582,2165.49,1.64876e+12,9.90014e+07,56751,903.948,4.1319e+07
2022-04-01 04:00:00+08:00,45582,45790,45519,45745.5,1550.11,1.64876e+12,7.07614e+07,40487,820.006,3.7424e+07
2022-04-01 05:00:00+08:00,45745.4,45850,45647,45757.2,887.899,1.64876e+12,4.06338e+07,30025,443.368,2.02892e+07
2022-04-01 06:00:00+08:00,45757.2,45820.3,45660.7,45787.8,1058.13,1.64877e+12,4.84091e+07,32731,506.214,2.31605e+07
2022-04-01 07:00:00+08:00,45787.8,45787.9,45200,45510.3,2889.28,1.64877e+12,1.31432e+08,62276,1271.26,5.7834e+07
2022-04-01 08:00:00+08:00,45510.3,45644.9,45351.7,45541.5,1838.2,1.64877e+12,8.37123e+07,51792,940.428,4.28273e+07


# Save pandas dataframe

In [13]:
def saveParquet(df, path):
    df.to_parquet(path)

In [14]:
path2save = "test.parquet"
saveParquet(df, path2save)

In [15]:
!ls .

binance-kline-data-download.ipynb  test.parquet


# Load pandas dataframe from file

In [16]:
df = pd.read_parquet(path2save)

In [17]:
df.head(10)

,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,take_buy_base_vol,take_buy_quote_vol
Timestamp,,,,,,,,,,
2022-04-01 00:00:00+08:00,46453.81,46492.65,45669.00,45688.80,6315.81153,1.648746e+12,2.905143e+08,143756.0,2779.39695,1.278521e+08
2022-04-01 01:00:00+08:00,45688.80,45891.94,45651.35,45820.00,2363.95325,1.648750e+12,1.082474e+08,64559.0,1225.24580,5.610397e+07
2022-04-01 02:00:00+08:00,45819.99,45962.08,45595.28,45957.27,1822.32955,1.648753e+12,8.345667e+07,54649.0,946.31150,4.334149e+07
2022-04-01 03:00:00+08:00,45957.27,45987.94,45500.00,45581.97,2165.48834,1.648757e+12,9.900144e+07,56751.0,903.94755,4.131903e+07
2022-04-01 04:00:00+08:00,45581.98,45790.00,45519.03,45745.46,1550.11357,1.648760e+12,7.076137e+07,40487.0,820.00555,3.742397e+07
2022-04-01 05:00:00+08:00,45745.45,45849.99,45647.01,45757.19,887.89890,1.648764e+12,4.063375e+07,30025.0,443.36795,2.028917e+07
2022-04-01 06:00:00+08:00,45757.19,45820.34,45660.68,45787.85,1058.12591,1.648768e+12,4.840908e+07,32731.0,506.21392,2.316047e+07
2022-04-01 07:00:00+08:00,45787.85,45787.86,45200.00,45510.34,2889.28168,1.648771e+12,1.314323e+08,62276.0,1271.26004,5.783396e+07
2022-04-01 08:00:00+08:00,45510.35,45644.86,45351.73,45541.50,1838.20071,1.648775e+12,8.371232e+07,51792.0,940.42832,4.282727e+07


In [18]:
df.tail(10)

,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,take_buy_base_vol,take_buy_quote_vol
Timestamp,,,,,,,,,,
2022-04-30 14:00:00+08:00,38703.01,38734.63,38646.69,38701.11,639.14028,1.651302e+12,2.472840e+07,26984.0,326.35256,1.262637e+07
2022-04-30 15:00:00+08:00,38701.11,38746.15,38634.24,38650.34,867.42132,1.651306e+12,3.355578e+07,32798.0,446.77611,1.728351e+07
2022-04-30 16:00:00+08:00,38650.35,38688.72,38536.66,38577.06,1542.29491,1.651309e+12,5.955541e+07,45935.0,661.41676,2.553948e+07
2022-04-30 17:00:00+08:00,38577.07,38644.03,38540.98,38622.50,881.80886,1.651313e+12,3.403453e+07,41209.0,416.72999,1.608292e+07
2022-04-30 18:00:00+08:00,38622.51,38658.72,38564.46,38566.49,804.52260,1.651316e+12,3.106105e+07,39497.0,412.95047,1.594269e+07
2022-04-30 19:00:00+08:00,38566.49,38636.21,38563.40,38580.01,727.99685,1.651320e+12,2.809666e+07,44632.0,363.71876,1.403722e+07
2022-04-30 20:00:00+08:00,38580.00,38610.00,38525.34,38536.23,767.52198,1.651324e+12,2.960570e+07,44769.0,316.39434,1.220373e+07
2022-04-30 21:00:00+08:00,38536.23,38638.24,38501.00,38506.33,1275.79399,1.651327e+12,4.919627e+07,39083.0,664.75244,2.563120e+07
2022-04-30 22:00:00+08:00,38506.32,38638.38,38211.00,38258.27,4427.80048,1.651331e+12,1.701338e+08,72208.0,1803.41623,6.931719e+07
